In [120]:
import pandas as pd 
import openpyxl
import yaml
from datetime import datetime, timedelta, date
import pulp
import xlsxwriter

In [121]:
proyecto="cabanaconde"

In [122]:
with open("./data/"+proyecto + '/datos.yaml', 'r',encoding='utf-8') as file:
    datos = yaml.safe_load(file)

print(datos)

{'ruta': 'Emp. R0405105(CABANACONDE)-CHOCO(LD.PROV. CASTILLA)', 'tramo': 'Emp. R0405105(CABANACONDE)-CHOCO(LD.PROV. CASTILLA)', 'longitud': 14064.0, 'fecha_inicio': {'day': 1, 'month': 5, 'year': 2025}, 'tiempo_ejecucion_dias': 240}


In [123]:
with open("./data/"+proyecto + '/partidas.yaml', 'r', encoding='utf-8') as file:
    partidas = yaml.safe_load(file)

print(partidas)

[{'codigo': 'MR100', 'descripcion': 'CONSERVACION DE CALZADA', 'actividades': [{'codigo': 'MR101', 'descripcion': 'Limpieza de Calzada', 'unidad_medida': 'Km', 'tablas_por_cuadrilla': 1, 'numero_cuadrillas': 1, 'total_tablas_por_actividad': 1, 'numero_dias': 45}, {'codigo': 'MR102', 'descripcion': 'Bacheo', 'unidad_medida': 'm2', 'tablas_por_cuadrilla': 1, 'numero_cuadrillas': 1, 'total_tablas_por_actividad': 1, 'numero_dias': 60}, {'codigo': 'MR103', 'descripcion': 'Desquinche', 'unidad_medida': 'm3', 'tablas_por_cuadrilla': 1, 'numero_cuadrillas': 1, 'total_tablas_por_actividad': 1, 'numero_dias': 5}, {'codigo': 'MR104', 'descripcion': 'Remoción de Derrumbes', 'unidad_medida': 'm3', 'tablas_por_cuadrilla': 1, 'numero_cuadrillas': 1, 'total_tablas_por_actividad': 1, 'numero_dias': 16}]}, {'codigo': 'MR200', 'descripcion': 'LIMPIEZA DE OBRAS DE ARTE', 'actividades': [{'codigo': 'MR201', 'descripcion': 'Limpieza de Cunetas', 'unidad_medida': 'm', 'tablas_por_cuadrilla': 1, 'numero_cuadr

## Funciones utiles

In [124]:
def concatenar_elementos(lista):
    elementos_como_texto = [str(e) for e in lista]
    return " ".join(elementos_como_texto)

In [125]:
def formatear_progresiva(distancia, decimales=0):
    """
    Convierte una distancia en metros a notación de progresiva.
    
    Parámetros:
    - distancia: int o float, la distancia en metros.
    - decimales: int, número de decimales a mostrar en la parte de los metros.
    
    Retorna:
    - str: progresiva en formato 'K+XXX' con los decimales indicados.
    """
    if not isinstance(distancia, (int, float)):
        raise ValueError("La distancia debe ser un número (int o float).")
    if not isinstance(decimales, int) or decimales < 0:
        raise ValueError("Los decimales deben ser un entero no negativo.")
    
    km = int(distancia) // 1000
    metros = distancia - (km * 1000)
    
    formato_metros = f"{metros:0.{decimales}f}".zfill(3 + (1 if decimales > 0 else 0) + decimales)
    return f"{km}+{formato_metros}"

In [126]:
from datetime import date

def mes_obra(fecha_inicio: date, mes: int) -> str:
    """
    Devuelve un string indicando el nº de mes de la obra respecto a la fecha de inicio.
    
    Args:
        fecha_inicio (date): Fecha en que empezó la obra (ej: date(2025,1,1)).
        mes (int): Número de mes (1–12) a evaluar.
    
    Returns:
        str: 
          - Si `mes` < mes de inicio: cadena vacía.
          - Si `mes` >= mes de inicio: "{n}º mes" donde n = (mes - mes_inicio + 1).
    """
    if not isinstance(fecha_inicio, date):
        raise TypeError("fecha_inicio debe ser un objeto date de datetime")
    if not (1 <= mes <= 12):
        raise ValueError("mes debe estar entre 1 y 12")
    
    # Calculamos cuántos meses han pasado desde el mes de inicio
    diff = mes - fecha_inicio.month + 1
    if diff < 1:
        return ""
    return f"{diff}º mes"


## Valores calculados relevantes

In [127]:
fecha_inicio=date(datos['fecha_inicio']['year'], datos['fecha_inicio']['month'], datos['fecha_inicio']['day'])
fecha_fin=fecha_inicio + timedelta(days=datos['tiempo_ejecucion_dias']-1) #Calculamos la fecha final sumando duracion_dias - 1 días (porque si el día 1 ya cuenta como primer día, solo sumamos los días restantes).
print("Fecha de inicio:", fecha_inicio)
print("Fecha de fin:", fecha_fin)

Fecha de inicio: 2025-05-01
Fecha de fin: 2025-12-26


## Programacion

In [128]:
h1=concatenar_elementos([
    "Programación inicial base de actividades",
    datos['fecha_inicio']['year'],
    "-",
    datos['tiempo_ejecucion_dias'],
    "días calendarios"]
)
print(h1)

Programación inicial base de actividades 2025 - 240 días calendarios


In [129]:
header_ruta=concatenar_elementos([
    'Ruta:',
    datos['ruta']])
header_tramo=concatenar_elementos([
    'Tramo:',
    datos['tramo']])
header_longitud=concatenar_elementos([
    'Longitud:',
    formatear_progresiva(datos['longitud']),
    "Km"]
    )

header_fecha_inicio=concatenar_elementos([
    'Fecha de inicio:',
    fecha_inicio.strftime("%d/%m/%Y")])
header_fecha_fin=concatenar_elementos([
    'Fecha de fin:',
    fecha_fin.strftime("%d/%m/%Y")])
header_tiempo_ejecucion=concatenar_elementos([
    'Tiempo de ejecución:',
    datos['tiempo_ejecucion_dias'],
    "días calendarios"])

In [130]:
print(header_fecha_fin)
print(header_fecha_inicio)

Fecha de fin: 26/12/2025
Fecha de inicio: 01/05/2025


In [131]:
# Actualización del script para incluir actividades con 0 días en la tabla final

# 1) PARÁMETROS DE ENTRADA
# -----------------------
json_actividades = partidas

fecha_inicio = datetime.strptime("2025-05-01", "%Y-%m-%d").date()
fecha_fin    = datetime.strptime("2025-12-26", "%Y-%m-%d").date()

# 2) EXTRACCIÓN DE ACTIVIDADES
# -----------------------------
actividades_all = {}
for grupo in json_actividades:
    for act in grupo["actividades"]:
        actividades_all[act["codigo"]] = act["numero_dias"]

# Separar actividades con >0 y aquellas con 0
actividades = {i: d for i, d in actividades_all.items() if d > 0}
actividades_zero = [i for i, d in actividades_all.items() if d == 0]

# 3) VALIDACIÓN DE CONSISTENCIA
# -----------------------------
total_act = sum(actividades.values())
total_periodo = (fecha_fin - fecha_inicio).days + 1
if total_act != total_periodo:
    raise RuntimeError(f"Error: suma de actividades ({total_act}) ≠ días del periodo ({total_periodo})")

# 4) GENERAR CALENDARIO POR MES
# -----------------------------
calendario = {}
hoy = fecha_inicio
while hoy <= fecha_fin:
    mes = hoy.strftime("%Y-%m")
    calendario[mes] = calendario.get(mes, 0) + 1
    hoy += timedelta(days=1)

# 5) MATRIZ IDEAL r[i][j]
# -----------------------
meses = list(calendario.keys())
S = sum(calendario.values())
r = {i: {m: actividades[i] * (calendario[m] / S) for m in meses} for i in actividades}

# 6) MODELO MIP CON PuLP
# -----------------------
prob = pulp.LpProblem("DistribucionDias", pulp.LpMinimize)

# Variables
x = pulp.LpVariable.dicts("x", (actividades.keys(), meses), lowBound=0, cat="Integer")
d = pulp.LpVariable.dicts("d", (actividades.keys(), meses), lowBound=0, cat="Continuous")

# Restricciones por actividad
for i, D_i in actividades.items():
    prob += pulp.lpSum(x[i][m] for m in meses) == D_i, f"fila_{i}"

# Restricciones por mes
for m, M_m in calendario.items():
    prob += pulp.lpSum(x[i][m] for i in actividades) == M_m, f"col_{m}"

# Desviaciones absolutas
for i in actividades:
    for m in meses:
        prob += x[i][m] - r[i][m] <= d[i][m]
        prob += r[i][m] - x[i][m] <= d[i][m]

# Objetivo
prob += pulp.lpSum(d[i][m] for i in actividades for m in meses)

# Resolver
print("Resolviendo MIP…")
prob.solve(pulp.PULP_CBC_CMD(msg=False))

# 7) EXTRAER SOLUCIÓN
data = {i: {m: int(pulp.value(x[i][m])) for m in meses} for i in actividades}

# Agregar actividades con 0 días
for i in actividades_zero:
    data[i] = {m: 0 for m in meses}

# 8) Mostrar tabla
# 1) Crear DataFrame y reordenar columnas cronológicamente
df = pd.DataFrame(data).T
meses = sorted(calendario.keys())      # ej. ['2025-05', '2025-06', …, '2025-12']
df = df[meses]

# 2) Total por actividad (filas)
df["TOTAL"] = df.sum(axis=1).astype(int)

# 3) Total por mes (columnas), incluyendo la columna "TOTAL"
column_totals = df.sum(axis=0).astype(int)
df.loc["TOTAL"] = column_totals

# 4) (Opcional) Reordenar filas para un orden específico
#    p.ej., manteniendo el orden original de actividades y luego "TOTAL"
orden_filas = list(actividades_all.keys()) + ["TOTAL"]
df = df.reindex(orden_filas)

print(df)



Resolviendo MIP…


       2025-05  2025-06  2025-07  2025-08  2025-09  2025-10  2025-11  2025-12  \
MR101        6        5        6        6        6        6        5        5   
MR102        8        7        8        8        7        8        7        7   
MR103        0        1        0        1        1        1        1        0   
MR104        2        2        2        2        2        2        2        2   
MR201        6        5        6        6        5        6        5        5   
MR202        0        0        0        0        0        0        0        0   
MR203        2        2        1        1        1        1        2        1   
MR204        0        0        0        0        0        0        0        0   
MR205        0        0        0        0        0        0        0        0   
MR206        0        0        0        0        0        0        0        0   
MR301        6        5        6        6        5        6        5        5   
MR401        0        1     

In [132]:
df.head(18)

,2025-05,2025-06,2025-07,2025-08,2025-09,2025-10,2025-11,2025-12,TOTAL
MR101,6,5,6,6,6,6,5,5,45
MR102,8,7,8,8,7,8,7,7,60
MR103,0,1,0,1,1,1,1,0,5
MR104,2,2,2,2,2,2,2,2,16
MR201,6,5,6,6,5,6,5,5,44
MR202,0,0,0,0,0,0,0,0,0
MR203,2,2,1,1,1,1,2,1,11
MR204,0,0,0,0,0,0,0,0,0
MR205,0,0,0,0,0,0,0,0,0
MR206,0,0,0,0,0,0,0,0,0


In [133]:
full_months = pd.period_range('2025-01', '2025-12', freq='M').strftime('%Y-%m').tolist()


In [134]:
df_full   = df.reindex(columns=full_months, fill_value=0)

### Escribiendo el documento excel

In [136]:
# 1. Crear el libro y la hoja
wb = xlsxwriter.Workbook("01_programacion_actividades.xlsx")
ws = wb.add_worksheet("programacion")

# 2. Definir formatos
header_fmt = wb.add_format(
    {
        "bold": True,
        "align": "center",
        "valign": "vcenter",
        "bg_color": "#D9E1F2",
        "border": 1,
    }
)

header2_fmt = wb.add_format(
    {
        "bold": True,
        "align": "left",
        "valign": "vcenter",
    }
)

table_header_fmt = wb.add_format(
    {"align": "center", "valign": "vcenter", "bg_color": "#FFFFFF", "border": 1}
)

cell_fmt = wb.add_format({"valign": "vcenter", "border": 1})



suma_fmt = wb.add_format(
    {"bold": True, "valign": "vcenter", "bg_color": "#D9E1F2", "border": 1}
)

formato_ajustar = wb.add_format(
    {
        "align": "center",
        "valign": "vcenter",
        "text_wrap": True,  # Ajustar texto
        "shrink": True,  # Reducir hasta ajustar
        "border": 1,
    }
)


month_fmt = wb.add_format({"align": "center", "bg_color": "#F2F2F2", "border": 1})
code_fmt = wb.add_format({"border": 1})
number_fmt = wb.add_format({"border": 1, "align": "center"})

# 3. Ajustar anchos de columna
ws.set_column("B:B", 8)  # Código
ws.set_column("C:C", 30)  # Actividad
ws.set_column("D:H", 8)  # Unid., Tab./Cuad., etc.
ws.set_column("H:T", 8)  # Meses
ws.set_column("U:U", 10)  # Nº días

# 4. Escribir título y combinar celdas
ws.merge_range("B2:U2", h1, header_fmt)

ws.merge_range("C3:J3", header_ruta, header2_fmt)
ws.merge_range("C4:J4", header_tramo, header2_fmt)
ws.merge_range("C5:J5", header_longitud, header2_fmt)

ws.merge_range("K3:O3", header_fecha_inicio, header2_fmt)
ws.merge_range("K4:O4", header_fecha_fin, header2_fmt)
ws.merge_range("K5:O5", header_tiempo_ejecucion, header2_fmt)

# 5. Escribir encabezados de tabla

ws.merge_range("B7:B10", "Código", table_header_fmt)
ws.merge_range("C7:C10", "Actividad", table_header_fmt)
ws.merge_range("D7:D10", "Unid.", table_header_fmt)
ws.merge_range("E7:E10", "Tab./Cuad.", table_header_fmt)
ws.merge_range("F7:F10", "Nº Cuad.", table_header_fmt)
ws.merge_range("G7:G10", "Tab. / Act.", table_header_fmt)
ws.merge_range("H7:H10", "Nº Dias", table_header_fmt)

ws.merge_range("I7:T7", "Meses", table_header_fmt)
ws.write("I8", "ENE", table_header_fmt)
ws.write("J8", "FEB", table_header_fmt)
ws.write("K8", "MAR", table_header_fmt)
ws.write("L8", "ABR", table_header_fmt)
ws.write("M8", "MAY", table_header_fmt)
ws.write("N8", "JUN", table_header_fmt)
ws.write("O8", "JUL", table_header_fmt)
ws.write("P8", "AGO", table_header_fmt)
ws.write("Q8", "SEP", table_header_fmt)
ws.write("R8", "OCT", table_header_fmt)
ws.write("S8", "NOV", table_header_fmt)
ws.write("T8", "DIC", table_header_fmt)

ws.merge_range("I9:K9", "ÉPOCA DE LLUVIAS", table_header_fmt)
ws.merge_range("L9:M9", "DESPUES DE LLUVIAS", table_header_fmt)
ws.merge_range("N9:Q9", "ÉPOCA SECA", table_header_fmt)
ws.merge_range("R9:T9", "ANTES DE LLUVIAS", table_header_fmt)

ws.merge_range("U7:U10", "N° días para ejecutar según contrato", formato_ajustar)

# VALORES CALCULADOS

ws.write("I10", mes_obra(fecha_inicio, 1), table_header_fmt)
ws.write("J10", mes_obra(fecha_inicio, 2), table_header_fmt)
ws.write("K10", mes_obra(fecha_inicio, 3), table_header_fmt)
ws.write("L10", mes_obra(fecha_inicio, 4), table_header_fmt)
ws.write("M10", mes_obra(fecha_inicio, 5), table_header_fmt)
ws.write("N10", mes_obra(fecha_inicio, 6), table_header_fmt)
ws.write("O10", mes_obra(fecha_inicio, 7), table_header_fmt)
ws.write("P10", mes_obra(fecha_inicio, 8), table_header_fmt)
ws.write("Q10", mes_obra(fecha_inicio, 9), table_header_fmt)
ws.write("R10", mes_obra(fecha_inicio, 10), table_header_fmt)
ws.write("S10", mes_obra(fecha_inicio, 11), table_header_fmt)
ws.write("T10", mes_obra(fecha_inicio, 12), table_header_fmt)

# 2. Prepara las filas
rows = []
for sección in partidas:
    # fila de sección (solo código y descripción, resto vacío)
    rows.append(
        {
            "Código": sección["codigo"],
            "Actividad": sección["descripcion"],
            "Unid.": "",
            "Tab. / Cuad.": "",
            "Nº Cuad.": "",
            "Tab. / Act.": "",
            "Nº Días": "",
        }
    )
    # filas de actividades
    for act in sección["actividades"]:
        rows.append(
            {
                "Código": act["codigo"],
                "Actividad": act["descripcion"],
                "Unid.": act["unidad_medida"],
                "Tab. / Cuad.": act["tablas_por_cuadrilla"],
                "Nº Cuad.": act["numero_cuadrillas"],
                "Tab. / Act.": act["total_tablas_por_actividad"],
                "Nº Días": act["numero_dias"],
            }
        )

# 8. Guardar el archivo

# 1. Define formatos
section_fmt = wb.add_format(
    {
        "bold": True,
        "border": 1,
    }
)
activity_indent = wb.add_format({"indent": 1, "border": 1})

# 2. Punto de inicio en B11
start_row, start_col = 10, 1  # B11 en 0‑based

# 3. Recorre rows y escribe
for i, row in enumerate(rows):
    r = start_row + i
    # si es fila de sección (unidad vacía), negrita; si no, formato normal
    is_seccion = row["Unid."] == ""
    fmt = section_fmt if is_seccion else None

    # columna B → Código
    ws.write(r, start_col + 0, row["Código"], fmt)
    # columna C → Actividad (con sangría si no es sección)
    ws.write(
        r,
        start_col + 1,
        row["Actividad"],
        section_fmt if is_seccion else activity_indent,
    )
    # resto de columnas D‑H
    ws.write(r, start_col + 2, row["Unid."], fmt)
    ws.write(r, start_col + 3, row["Tab. / Cuad."], fmt)
    ws.write(r, start_col + 4, row["Nº Cuad."], fmt)
    ws.write(r, start_col + 5, row["Tab. / Act."], fmt)
    ws.write(r, start_col + 6, row["Nº Días"], fmt)

# 4. (Opcional) Ajusta anchos
ws.set_column(start_col + 0, start_col + 0, 12)  # Código
ws.set_column(start_col + 1, start_col + 1, 40)  # Actividad
ws.set_column(start_col + 2, start_col + 6, 12)  # resto

## Llenar la tabla con los datos de df_full

# 2) Lista de códigos en el orden de tu hoja (incluye títulos y detalles)
sheet_codes = [
    "MR100",  # título
    "MR101",
    "MR102",
    "MR103",
    "MR104",
    "MR200",  # otro título
    "MR201",
    "MR202",
    "MR203",
    "MR204",
    "MR205",
    "MR206",
    "MR300",  # otro título
    "MR301",
    "MR400",  # otro título
    "MR401",
    "MR500",  # otro título
    "MR501",
    "MR600",  # otro título
    "MR601",
    "MR700",  # otro título
    "MR701",
    "MR702",
]

start_row = 11  # fila donde empiezan (1-based)
start_col = 9  # col I → 9 (1-based)
for i, code in enumerate(sheet_codes):
    row = start_row - 1 + i  # 0-index
    if code in df_full.index:
        for j, mes in enumerate(full_months):
            col = start_col - 1 + j
            val = df_full.at[code, mes]
            ws.write(row, col, val)
    # si no está en df_full (fila de título), no escribimos nada → queda en blanco

# LLENANDO LA TABLA CON LOS TOTALES

filas = [12, 13, 14, 15, 17, 18, 19, 20, 21, 22, 24, 26, 28, 30, 32, 33]

# 3. Bucle para escribir la fórmula SUM(I#:T#) en la columna U de cada fila
for fila in filas:
    celda_destino = f"U{fila}"  # e.g. 'U12', 'U13', …
    formula = f"=SUM(I{fila}:T{fila})"  # e.g. '=SUM(I12:T12)'
    ws.write_formula(celda_destino, formula,suma_fmt)

# 2. Generar la lista de columnas de la 'I' a la 'U'
#    Usamos los códigos ASCII para pasar de 'I' (73) a 'U' (85).
columnas = [chr(c) for c in range(ord("I"), ord("U") + 1)]

# 3. Bucle para insertar SUMA en fila 34 de cada columna
for col in columnas:
    celda_destino = f"{col}34"  # e.g. 'I34', 'J34', …
    rango = f"{col}12:{col}33"  # e.g. 'I12:I33'
    formula = f"=SUM({rango})"  # '=SUM(I12:I33)'
    ws.write_formula(celda_destino, formula,suma_fmt)

# AGREGANDO BORDE DE LA TABLA

ws.conditional_format("B11:U33", {"type": "no_errors", "format": cell_fmt})

wb.close()